# Batches v2

In [1]:
import tensorflow as tf

def process_path(paths, labels):
    features = []
    # load waveforms in memory
    for path in paths:
        raw_audio = tf.io.read_file(path)
        waveform, sr = tf.audio.decode_wav(raw_audio)
        features.append(waveform)

#     tf.print(features[0], len(features[0]), len(features[1]))
    features = tf.keras.preprocessing.sequence.pad_sequences(features, dtype='float', padding='post', truncating='post')
#     tf.print(features[0], len(features[0]), len(features[1]))
#     tf.print(features)
    return features, labels

def tf_function_wrapper(features, labels):
    return tf.py_function(func=process_path, inp=[features['filename'], labels], Tout=tf.float32), labels

librivox_paths_ds = tf.data.experimental.make_csv_dataset(
    "librivox-test-clean-wav.csv", batch_size=2,
    shuffle=False, label_name="transcript")

librivox_ds = librivox_paths_ds.map(tf_function_wrapper)

for feature_batch, labels in librivox_ds.take(1):
    tf.print(feature_batch)

    

[[[0.000732421875]
  [0.000823974609]
  [0.00173950195]
  ...
  [0]
  [0]
  [0]]

 [[-0.00778198242]
  [-0.00921630859]
  [-0.00909423828]
  ...
  [-0.00262451172]
  [-0.00671386719]
  [-0.0113830566]]]


In [67]:
import tensorflow as tf
librivox_paths_ds = tf.data.experimental.make_csv_dataset(
    "librivox-train-clean-100-wav.csv", batch_size=2,
    shuffle=False, label_name="transcript")

def filter_fn(x, y):
    comparison = tf.math.less(tf.strings.length(y), tf.constant(100))
    return tf.math.count_nonzero(comparison) == len(y)

librivox_paths_ds = librivox_paths_ds.filter(filter_fn)
for features, labels in librivox_paths_ds.take(1):
    tf.print(tf.strings.length(labels))
# list(librivox_paths_ds.take(1).as_numpy_iterator()) 

[51 45]


In [5]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3]) 
dataset = dataset.filter(lambda x: x < 3) 
list(dataset.as_numpy_iterator()) 

# `tf.math.equal(x, y)` is required for equality comparison 
def filter_fn(x): 
    return tf.math.equal(x, 1) 
dataset = dataset.filter(filter_fn) 
list(dataset.as_numpy_iterator()) 

[1]

In [51]:
for feature_batch in librivox_ds.take(1):
    tf.print(feature_batch[0][1])
    for key, valeu in feature_batch.numpy():
        print(key)
#     for key, valeu in feature_batch.numpy():
#         print(value)

[[-0.00778198242]
 [-0.00921630859]
 [-0.00909423828]
 ...
 [-0.00262451172]
 [-0.00671386719]
 [-0.0113830566]]


AttributeError: 'tuple' object has no attribute 'numpy'

In [91]:
import tensorflow as tf
t = tf.constant([[1, 2, 3], [4, 5, 6]])
paddings = tf.constant([[0, 0], [0, 2]])
# 'constant_values' is 0.
# rank of 't' is 2.
a = tf.pad(t, paddings, "CONSTANT")  # [[0, 0, 0, 0, 0, 0, 0],
                                 #  [0, 0, 1, 2, 3, 0, 0],
                                 #  [0, 0, 4, 5, 6, 0, 0],
                                 #  [0, 0, 0, 0, 0, 0, 0]]
a

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[1, 2, 3, 0, 0],
       [4, 5, 6, 0, 0]], dtype=int32)>

In [3]:
from soundfile import read
path = '/Users/gabriel.t.nishimura/projects/masters/ctc/data_dir/LibriSpeech/test-clean-new/61-70970-0000.wav'
read(path)

(array([ 0.00073242,  0.00082397,  0.0017395 , ..., -0.00631714,
        -0.00570679, -0.00668335]),
 16000)

In [5]:
import tensorflow as tf
raw_audio = tf.io.read_file(path)
waveform, sample_rate = tf.audio.decode_wav(raw_audio)
transwav = tf.transpose(waveform[0])
stfts = tf.signal.stft(transwav, frame_length=1024, frame_step=256, fft_length=1024)
spectrograms = tf.abs(stfts)

num_spectrogram_bins = stfts.shape[-1]
lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, 80

linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
  num_mel_bins, num_spectrogram_bins, sample_rate, lower_edge_hertz,
  upper_edge_hertz)

mel_spectrograms = tf.tensordot(
  spectrograms, linear_to_mel_weight_matrix, 1)

mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(
  linear_to_mel_weight_matrix.shape[-1:]))

# Compute a stabilized log to get log-magnitude mel-scale spectrograms.
log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

# Compute MFCCs from log_mel_spectrograms and take the first 13.
mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrograms)
tf.print(mfccs)

InvalidArgumentError: Input dimension 1 must have length of at least 160 but got: 80 [Op:RFFT] name: mfccs_from_log_mel_spectrograms/dct/rfft/

In [25]:
import tensorflow as tf

def process_path(paths):
    features = []
    for path in paths:
        raw_audio = tf.io.read_file(path) # load file
        waveform, sample_rate = tf.audio.decode_wav(raw_audio) # load waveforms in memory
        transwav = tf.transpose(waveform) # shanenigans to stft
        stfts = tf.signal.stft(transwav, frame_length=1024, frame_step=256, fft_length=1024)
        spectrograms = tf.abs(stfts)

        num_spectrogram_bins = stfts.shape[-1]
        lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, 80

        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
          num_mel_bins, num_spectrogram_bins, sample_rate, lower_edge_hertz,
          upper_edge_hertz)

        mel_spectrograms = tf.tensordot(
          spectrograms, linear_to_mel_weight_matrix, 1)

        mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(
          linear_to_mel_weight_matrix.shape[-1:]))

        # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
        log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

        # Compute MFCCs from log_mel_spectrograms
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrograms)
        features.append(mfccs[0])

    # pad different sized mfccs
    features = tf.keras.preprocessing.sequence.pad_sequences(features, dtype='float', padding='post', truncating='post')
    return features

def process_labels(labels):
    
    return labels
    

def tf_function_wrapper(features, labels):
    features = tf.py_function(func=process_path, inp=[features['filename']], Tout=tf.float32)
    labels = tf.py_function(func=process_labels, inp=[labels], Tout=tf.int32)
    return features, labels

librivox_paths_ds = tf.data.experimental.make_csv_dataset(
    "librivox-test-clean-wav.csv", batch_size=4,
    shuffle=False, label_name="transcript")

librivox_ds = librivox_paths_ds.map(tf_function_wrapper)

for feature_batch, labels in librivox_ds.take(1):
    tf.print(labels)

    

InvalidArgumentError: ValueError: Tensor conversion requested dtype int32 for Tensor with dtype string: <tf.Tensor: shape=(4,), dtype=string, numpy=
array([b"young fitzooth had been commanded to his mother's chamber so soon as he had come out from his converse with the squire",
       b"there befell an anxious interview mistress fitzooth arguing for and against the squire's project in a breath",
       b'most of all robin thought of his father what would he counsel',
       b'if for a whim you beggar yourself i cannot stay you'],
      dtype=object)>
Traceback (most recent call last):

  File "/Users/gabriel.t.nishimura/miniconda3/envs/usp/lib/python3.6/site-packages/tensorflow_core/python/ops/script_ops.py", line 234, in __call__
    return func(device, token, args)

  File "/Users/gabriel.t.nishimura/miniconda3/envs/usp/lib/python3.6/site-packages/tensorflow_core/python/ops/script_ops.py", line 141, in __call__
    self._convert(ret, dtype=self._out_dtypes[0]), device_name)

  File "